In [15]:
# Importing the libraries 

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

# Ignore harmless warnings 

import warnings 
warnings.filterwarnings("ignore")

# Set to display all the columns in dataset

pd.set_option("display.max_columns", None)

# Import psql to run queries 

import pandasql as psql

In [2]:
# load the Concrete dataset 

concrete = pd.read_csv(r"E:\win7\files\13,14-08-22\Concrete_Data_V1.0.csv", header=0)

# copy the file to back-up file

concrete_bk = concrete.copy()

# display first 5 records

concrete.head()

,Cement,Slag,Flyash,Water,SuperPlasticizer,CoarseAggregate,FineAggregate,Age,CSinMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Display concrete data information

concrete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Cement            1030 non-null   float64
 1   Slag              1030 non-null   float64
 2   Flyash            1030 non-null   float64
 3   Water             1030 non-null   float64
 4   SuperPlasticizer  1030 non-null   float64
 5   CoarseAggregate   1030 non-null   float64
 6   FineAggregate     1030 non-null   float64
 7   Age               1030 non-null   int64  
 8   CSinMPa           1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [4]:
# Display the columns in concrete dataset

concrete.columns

Index(['Cement', 'Slag', 'Flyash', 'Water', 'SuperPlasticizer',
       'CoarseAggregate', 'FineAggregate', 'Age', 'CSinMPa'],
      dtype='object')

In [5]:
# Prepare cols1 for scaling

cols1 = ['Cement', 'Slag', 'Flyash', 'Water', 'SuperPlasticizer', 'CoarseAggregate', 'FineAggregate', 'Age']

#cols1 = ['Cement', 'Slag', 'Flyash', 'Water', 'SuperPlasticizer', 'CoarseAggregate', 'FineAggregate', 'Age']

In [6]:
# Identify the independent and Target variables

IndepVar = []
for col in concrete.columns:
    if col != 'CSinMPa':
        IndepVar.append(col)

TargetVar = 'CSinMPa'

x = concrete[IndepVar]
y = concrete[TargetVar]

In [7]:
# Split the data into train and test

from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=143)

# Display the shape of the train_data and test_data

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((721, 8), (309, 8), (721,), (309,))

In [8]:
# Scaling the features by using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

mmscaler = MinMaxScaler(feature_range=(0, 1))

x_train[cols1] = mmscaler.fit_transform(x_train[cols1])
x_train = pd.DataFrame(x_train)

x_test[cols1] = mmscaler.fit_transform(x_test[cols1])
x_test = pd.DataFrame(x_test)

In [9]:
#train the algorithm and build the model with train dataset

from sklearn.ensemble import ExtraTreesRegressor

# Create model object

ModelETR = ExtraTreesRegressor()



ModelETR.fit(x_train, y_train)

y_pred=ModelETR.predict(x_test)

# Evaluation metrics for Regression analysis

from sklearn import metrics

print('Mean Absolute Error (MAE):',  round(metrics.mean_absolute_error(y_test, y_pred),3))  
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, y_pred),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3))
print('R2_score:', round(metrics.r2_score(y_test, y_pred),6))
print('Root Mean Squared Log Error (RMSLE):', round(np.log(np.sqrt(metrics.mean_squared_error(y_test,y_pred))),3))

# Define the function to calculate the MAPE - Mean Absolute Percentage Error

def MAPE (y_test, y_pred): 
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# Evaluation of MAPE 

result = MAPE(y_test, y_pred)
print('Mean Absolute Percentage Error (MAPE):', round(result, 3), '%')
#Sruthi G21:55
#print('Root Mean Squared Log Error (RMSLE):', round(np.log(np.sqrt(metrics.mean_squared_error(y_test,y_pred))),3))

# Calculate Adjusted R squared values 

r_squared = round(metrics.r2_score(y_test, y_pred),6)
adjusted_r_squared = round(1 - (1-r_squared)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
print('Adj R Square: ', adjusted_r_squared)

Mean Absolute Error (MAE): 4.295
Mean Squared Error (MSE): 35.95
Root Mean Squared Error (RMSE): 5.996
R2_score: 0.86278
Root Mean Squared Log Error (RMSLE): 1.791
Mean Absolute Percentage Error (MAPE): 13.239 %
Adj R Square:  0.861705


In [11]:
# Predict the values with ET algorithm

y_predF = ModelETR.predict(x_test)

In [12]:
Results = pd.DataFrame({'CSinMPa_A':y_test, 'CSinMPa_P_F':y_predF})

# Merge two Dataframes on index of both the dataframes

ResultsFinal = concrete_bk.merge(Results, left_index=True, right_index=True)
ResultsFinal.sample(10)

,Cement,Slag,Flyash,Water,SuperPlasticizer,CoarseAggregate,FineAggregate,Age,CSinMPa,CSinMPa_A,CSinMPa_P_F
375,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,14,25.62,25.62,25.9506
200,190.7,0.0,125.4,162.1,7.8,1090.0,804.0,14,21.06,21.06,21.4044
147,388.6,97.1,0.0,157.9,12.1,852.1,925.7,56,55.20,55.20,61.3155
462,172.4,13.6,172.4,156.8,4.1,1006.3,856.4,100,37.68,37.68,40.9170
693,153.0,102.0,0.0,192.0,0.0,888.0,943.1,28,17.96,17.96,20.9501
883,149.0,236.0,0.0,176.0,13.0,847.0,893.0,28,32.96,32.96,34.8241
569,381.4,0.0,0.0,185.7,0.0,1104.6,784.3,28,22.49,22.49,33.3163
364,214.9,53.8,121.9,155.6,9.6,1014.3,780.6,3,18.02,18.02,15.1908
621,307.0,0.0,0.0,193.0,0.0,968.0,812.0,180,34.49,34.49,33.5830
951,152.7,144.7,0.0,178.1,8.0,999.7,822.2,28,19.01,19.01,22.9487


In [13]:
# Calculate the %of Error

ResultsFinal['%Error'] = round(((ResultsFinal['CSinMPa_A']-ResultsFinal['CSinMPa_P_F'])/ResultsFinal['CSinMPa_A'])*100,3)

In [14]:
# Display the results

ResultsFinal.sample(5)

,Cement,Slag,Flyash,Water,SuperPlasticizer,CoarseAggregate,FineAggregate,Age,CSinMPa,CSinMPa_A,CSinMPa_P_F,%Error
90,389.9,189.0,0.0,145.9,22.0,944.7,755.8,3,40.60,40.60,33.9507,16.378
801,252.0,0.0,0.0,185.0,0.0,1111.0,784.0,28,19.69,19.69,18.6192,5.438
901,144.0,170.0,133.0,192.0,8.0,814.0,805.0,28,29.87,29.87,30.5595,-2.308
466,190.3,0.0,125.2,166.6,9.9,1079.0,798.9,100,33.56,33.56,42.8029,-27.541
426,173.8,93.4,159.9,172.3,9.7,1007.2,746.6,14,29.55,29.55,29.1294,1.423
